# Thinking Probabilistically 

In [ ]:
#| echo: false
import polars as pl 
import polars.selectors as cs
from scipy.special import binom, beta
import preliz as pz
from great_tables import GT 

## A Probability Primer for Bayesian Practitioners 

###  Sample space and events 

Lets say we go out and ask 3 random respondents a series of three questions and record their answers. There are lots of ways we could represent. In @tbl-samp-space I remix the example in the book a bit but the core idea is the same. The *sample space* is the set of all possible outcomes. 

In [ ]:
#| echo: false
#| label: tbl-samp-space
sample_space = pl.DataFrame({
    'Person 1': ['yes', 'yes', 'yes'],
    'Person 2': ['no', 'no', 'no'],
    'Question': ['1', '2', '3']
}).unpivot(on = cs.contains('Person'), index = 'Question', variable_name='Individual', value_name='Response')

GT(sample_space)

In our case, we are just talking about the order of each person's responses. An event is a subset of the feature space, so we can think of it as the order in which people respond to the questions. So there are eight possible *combinations* of answer orders. For brevity, I only include two of those events. If we wanted to know how likely it is that we observe an event, we can express it as a simple proportion. 

$$
P(Event) = \frac{\text{event we are interested in}}{\text{total number of events}}
$$

So if we are interested in the all yes event, it is just `{python} 1/8`. If we make the assumption that the events are independent of each other then the probability of observing an event where the respondent answers all yes and another respondent answers all no than we can represent that as:

$$
P(\text{All Yes and No}) = \frac{1}{8} + \frac{1}{8} = \frac{2}{8} = 0.25
$$


Fast forward a bit the three axioms of probability are: 

1. The probability of an event is an even is a non-negative real number 
2. The probability of P(sample space) = 1
3. If 𝐴1, 𝐴2,… are disjoint events, meaning they cannot occur simultaneously then
𝑃(𝐴1, 𝐴2, …) = 𝑃(𝐴1) + 𝑃(𝐴2) + …

### Random Variables 


A random variable is effectively a function that maps the sample space the real numbers. So we can think of that as. Doing something akin to: 


In [ ]:
#| echo: false
ex = sample_space.with_columns(
    pl.when(pl.col('Response') == 'yes').then(1).otherwise(0).alias('Mapping Examp')
)

GT(ex)

The author uses the example of a set of Python code. So if the take example from `pl.when(thing_happens).then(do this).otherwise(do that)` this is an abstraction over a set of Rust code that takes our inputs and then maps them to our desired output. We could theoretically write out all this Rust code every single time. But we package it in a nice set of functions to make the end user's code cleaner and reusable we make some functions to do this. 

Random variables play this similar role in stats where we have some inputs like responses to a survey or temperature that get mapped onto some process. There is no randomness when moving from "yes" to "one", but there *is* some randomness when we go to the survey the third and fourth person to get their event. 

### Discrete random variables and their distributions

We generally don't care about the kidns of problems that get presented in these refreshers. The probability of 3 inidividuals answering all yes or the probability of getting a one on the dice. In general we are more intersted in getting a list of probabilities for all possible answers. We can then get the possibilities of any sort of event we would care about. As a simple heuristic the list of the probabilities are effectively just this list of probabilities. So if we wanted to get the proportion of heads of 1000 flipped coins we could do this. 

In [ ]:
#| label: fig-coin-flips
#| echo: false
import numpy as np 
import matplotlib.pyplot as plt 

flips = np.random.choice(['heads', 'tails'], size = 1000, replace = True)

coin_flips = pl.DataFrame({
    'heads_proportion': np.cumsum(flips == 'heads')/np.arange(1, 1000 + 1),
    'flip_number': np.arange(1, 1000 + 1)
})

plt.figure(figsize=(10,6))
plt.hist(coin_flips['heads_proportion'])


In this case the probability distribution of a coin flip is a Bernouli distribution. When we have multiple Bernouli trials we can stack them and make a binomial. One thing about these functions is that they have, for lack of a better term, shape parameters. For a normal those are our mean and standard deviation for something like the beta binomial in `rbinom()` it is `rbinom(shape1 = 2, shape2 = 9)` which changes where the most likely outcomes are. 


In [ ]:
#| code-fold: true

np.random.seed(1994)

from cycler import cycler
default_cycler = cycler(color=["#000000", "#6a6a6a", "#bebebe", "#2a2eec"])

plt.rc('axes', prop_cycle=default_cycler)
plt.rc('figure', dpi=600)

n = 5

x = np.arange(0,5)
fig, axes = plt.subplots(2,2, figsize = (10,6), sharey= True, sharex = True)
axes = np.ravel(axes)


for ax, (alpha, bt) in zip(axes, ((1,1), (6,4),(4,6), (20,20))):
    dist_pmf = binom(n, x) * (beta(x+alpha, n-x+bt) / beta(alpha, bt))
    ax.vlines(x, 0, dist_pmf, colors = 'C0', lw=4)
    ax.set_title(f"α={alpha}, β={bt}")
    ax.set_xticks(x)
    ax.set_xticklabels(x + 1)

fig.text(0.52, -0.04, "x", fontsize=18)
fig.text(-0.04, 0.4, "P(X=x)", fontsize=18, rotation="vertical")

While this is helpful for us as we are toying around with setting priors or as pedagogy. The thing that is doing the heavy lifting of mapping things to the like P(thing) = 0.6 is the *probability mass function*. This works for discrete data because you can't have 1.5 heads. I eschew including it but one really helpful thing to help us wrap our heads around what changing the shape parameters of our distribution are is using the interactive plots. 


In [ ]:
bb = pz.BetaBinomial(alpha = 10, beta = 10, n = 6)

bb.plot_interactive(figsize = (10,6), pointinterval=False, xy_lim =(20,20))


### Continous random variables and their distributions 

Continous values are a bit different. They, theoretically, can go go from $-\infty$ to $\infty$ so the mapping has to work a little bit differently. Instead of a probalility mass function where we are moving from heads to 1 we have to effectively create *bins* and we count how many values fall in each bin. If we take a simple example of going from -1 to 1 we can get a huge number of points. So the actual probablity of getting exactly one in this context is `{python} 1/100` making it somewhat useless to do probablility like this. Hence why we take this density approach. 


### Cumulative distribution function

Finally we have the CDF to characterize distributions. The CDF of a random variable is kind of just the idea that we can take the mappings of a PMF or a PDF and then just map them onto a fairly simple question. What is the probability we get a point as big or less than the point we drew from that distribution. 

### Conditional Probablities 

From the highest level of abstraction all our models are really just fancy conditional probability machines. Given an outcome variable how does it change if we observe these 1 to a million things. To keep it more manageable than lets just consider the bivariate case. If we are trying to understand how something about how good an NFL team is we might want to look at their EPA and condition on adjusted games lost. The more games that a team's best players lose the worse their EPA is going to be. By knowing something about EPA we know a little bit something about their 

We would anticipate that the higher a team's EPA is the less adjusted games lost they have. If we plot the marginal distributions we can see that. Quickly to compute the marginal distribution of AGL we are going to take the p(AGL, EPA) and then we average over EPA. In @fig-marginals we can see that a bit more clearly. The dashed lines are the conditional probabilities when we sample from the p(AGL, EPA).


In [ ]:
#| label: fig-marginals
#| code-fold: true

# take steps of 0.1 from -4 to 4 effectively just seq(-4, 4, by = 0.1)
x, y = np.mgrid[-4:4:.01, -4:4:.01]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y

## fill the empty 3d array 
pos[:, :, 0] = x; pos[:, :, 1] = y

## initialize a multivariate normal 

rv = pz.MvNormal([0, 0], [[1, 0.8], 
                          [0.8, 1]])


x_value = pos[:, :, 0][:,0]
x_density = rv.pdf(pos)

left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width + 0.02

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# this is effectively making small multiples
# that we then place in the positions defined above 
_, axes_arr = plt.subplots(1, 3, figsize=(8, 8))

ax_joint, ax_x, ax_y = axes_arr
ax_joint.set_position(rect_scatter)
ax_x.set_position(rect_histx)
ax_y.set_position(rect_histy)

ax_joint.imshow(x_density, cmap='cet_gray_r', origin='lower', extent=[-3, 3, -3, 3])

# these are effectively arbitrary as long as they are within the grid
ax_joint.plot(x_value, x_density[400]*2, 'k:', lw=2)
ax_joint.plot(x_value, x_density[500]*2+1, 'k:', lw=2)
ax_joint.plot(x_value, x_density[300]*2-1, 'k:', lw=2)


ax_x.fill_between(x_value, x_density.sum(1), color = 'C2')
ax_y.fill_between(x_value, x_density.sum(1), color = 'C2')

for ax in [ax_joint, ax_x, ax_y]:
    ax.grid(False)
    ax.set_facecolor('w')
    ax.set_xticks([])
    ax.set_yticks([])
ax_joint.set_xlim(-3, 3)
ax_joint.set_ylim(-3, 3)
ax_x.set_xlim(-3, 3)
ax_y.set_ylim(-3, 3)
ax_x.set_xlim(-3, 3)
ax_joint.set_ylabel('$B$', rotation=0, labelpad=20, fontsize=18)
ax_joint.set_xlabel('$A$', fontsize=18)

ax_joint.text(-2.5, 2.5, '$p(EPA, AGL)$', fontsize=18, color='k', weight='medium')
ax_y.text(10, 0, '$p(GPL)$', fontsize=18, color='k', weight='medium')
ax_x.text(-0.2, 15, '$p(EPA)$', fontsize=18, color='k', weight='medium')
ax_joint.text(1, -2, ' ... $p(EPA \mid AGL)$', fontsize=18, color='k', weight='medium')

### Expected Values 

Lets say that we have a discrete random variable **X**. We can compute its expected value as: 

$$
E(X) = \Sigma_{x} xP(X=x)
$$

Which is just another way of expressing the average. This is one version of an expected value. Instead of doing the usual hand calculating of a mean. The variance is also another form of an expected value. Sometimes we refer to these as the moment of a distribution. In a Gaussian distribution we also have something called kurtosis which tells us how much stuff is in the tails of the distribution. We also has skewness which shifts where the masses are so we could do something like this. 


In [ ]:
_, ax = plt.subplots(2,2, figsize = (12,6), sharex=True)

pz.Normal(0, 1).plot_pdf(ax=ax[0, 0], moments=["m", "d", "s", "k"], legend="title")
pz.BetaScaled(1.1, 1.1, -4, 4).plot_pdf(ax=ax[0, 1], moments=["m", "d", "s", "k"], legend="title")
pz.SkewNormal(0, 1, 3).plot_pdf(ax=ax[1, 0], moments=["m", "d", "s", "k"], legend="title")
pz.StudentT(3, 0, 1).plot_pdf(ax=ax[1, 1], moments=["m", "d", "s", "k"], legend="title", support=(-7, 7))

## Bayes Theorum 

Lets fastforward a bit so we can express Bayes' rule as 

$$
\overbrace{p(hypothesis | data)}^{Posterior} = \frac{\overbrace{p(data | hypothesis)}^{likelihood}\overbrace{p(hypothesis)}^{prior}}{\underbrace{P(data)}_{marginal likelihood}}
$$


- **Prior**: The prior is a reflection of what we know about the hypothesis prior to ever seeing the data. There are a lot of ways to think of priors which is always something that is a bit difficult to wrangle our head around. In a machine learning framework we could think of a prior as a form of regularization. We don't actually think every value from 0-1 are equally likely in a model of EPA. If we draw a QB from a Beta distribution it is unlikely that we are going to draw a QB has an EPA of 0.9.^[For reference the EPA/Play leader in 2024 was Jared Goff with 0.29 and the leader in 2023 was Brock Purdy with 0.338/] From a classical statistical perspective we can really just think of the prior as a more apparent assumption we are making. 

- **Likelihood**: The likelihood is the way we introduce data to our analysis. So in the model it is really just "what is the probability of the data conditional on the hypothesis". So in an OLS the likelihood is just the normal pdf. 

- **Posterior**: The posterior practically is just the product of the likelihood and the prior. So to put this a bit more concretly: it is the probability of the data conditional on the hypothesis multiplied by what believe before seeing the data. This is itself a probability distribution. A conjugate prior of a likelihood is a prior that, when used in combination with a given likelihood, returns a posterior with the same functional form as the prior

## Single Parameter inference

### The Coin-flipping problem 

Lets say we are not the biggest joy to have at parties. We corner some unlucky party goers and we want to know if the quarter is fair coin. This could be important if we want to model the bias of a coin especially if we are like trying to figure out how to game the coin flip for a game or something. Really this is just a pedagocial example so no need to dwell on it that much. 

### Choosing A likelihood 

If we assume that these trials are independent. Meaning if its just me and a friend flipping coins we find in the street we can assume that these are independent. For this we can safely use the Binomial likelihood. To make this a little more concrete lets write out the likelihood with our toy example 

$$
p(\text{getting heads} | \Theta) = \frac{\text{Number of Tosses}!}{\text{getting heads}!(Tosses - \text{getting heads})!}\Theta^{\text{Getting Heads}} (1 - \Theta)^{\text{Tosses - \text{Getting Heads}}}
$$


In [ ]:
n_trials = [1,2, 5]
getting_heads = [0.25, .5, 0.75]


_,axes = plt.subplots(3,3, figsize = (12,6), sharex = True, sharey = True)
for i, n in enumerate(n_trials):
    for j, p in enumerate(getting_heads):
        pz.Binomial(n, p).plot_pdf(legend = 'title', ax = axes[i, j])


When we look at the plots as the number of trials grow we get more or less a probability of approx 50/50. When we change the probability to 0.5 and a t one trial it fits our mental model fairly well. While this is not always feasible it definitely helps. 

### Choosing a prior 

A natural choice for our prior is the the beta distrbitution. It is bounded between 0 and 1 which helps in our coin flip problem but is especially helpful for proportions. We would have to play with our shape parameters a bit but. A mostly symetric shape around .5 makes the most sense a priori. We are not going to entirely rule out a biased coin in any direction but we feel pretty confident that over like 20 trials the proportion of heads is going to be closer to .5 than it is 1 or 0. The big benefit computationally is that the Beta distribution and the binomial are that it returns a posterior that is the same functional form of the prior making it easier to explore the posterior. This was a much bigger deal in olden times.^[By olden times we are really talking closer to like your childhood.]

### How to choose priors 

As a Bayesian we do want the data to speak for itself. But we don't have to pretend like we don't know anything about the data. We really want to shoot for weakly informative priors. We know that a player or an offense generating close to 1 EPA/play is next to impossible in the NFL. Like it could theoretically happen but that would likely require us to take a top 5 offense in the NFL in the past 5-10 years and transport it back to when players were moonlighting in the offseason. However, we are probably not likely to see that or would not be a fun excercise to undertake. Instead we can impose some weakly informative priors to encourage our models to explore a reasonable set of bounds of the posterior distribution. 


One helpful way we can start to do prior tuning is using `preliz`. In some respects we could theoretically just hand derive it but that is not a good use of our time. In the plot below we are just saying 90% of values should be between 0.1 and 0.7. This will then spit out the priors in the legend. We can then take this prior and feed it over to our modeling packages which is super nice. Since sometimes hand deriving logit priors and beta priors is supertedious. 


In [ ]:
dist = pz.Beta()

pz.maxent(dist, 0.1, 0.7, 0.9)


pz.Beta(alpha = 2.47, beta = 3.61).to_pymc()

## Excercises 

These will be a bit briefer than the ones in the book. 

3. Use PreliZ to explore different parameters for the BetaBinomial and Gaussian distribution. 


In [ ]:
# _, ax = plt.subplots(2,2, figsize = (12,6), sharex  )